In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing modules
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [ ]:
# Importing dataset
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

# Showing dataset
print(train.head, test.head, sep="\n")

In [ ]:
# Features selected for model
features = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF']
encoded_features = ['MSZoning', 'Neighborhood', 'BldgType', 'HouseStyle', 'Exterior1st', 'ExterQual', 'KitchenQual', 'GarageType', 'SaleType', 'SaleCondition']

# Extracting those features
Y = train['SalePrice']
train = train[features+encoded_features]
X = train
test = test[features+encoded_features]

# Encoding the features with string values
for column in encoded_features:
    le = LabelEncoder()
    train[column] = le.fit_transform(train[column].astype(str))
    test[column] = le.fit_transform(test[column].astype(str))

# Handling missing values
train.dropna()
test.dropna()

# Showing processed data
print(train)
print(test)

In [ ]:
# Train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 53)

# Showing dataset
print(len(X_train), len(X_test))

In [ ]:
# Model define
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05)

# Model training
model.fit(X_train, Y_train, early_stopping_rounds=5, eval_set=[(X_test, Y_test)], verbose=True)

In [ ]:
# Model predictions
predictions = model.predict(X_test)

# Model metrics
mae = mean_absolute_error(Y_test, predictions)
mse = mean_squared_error(Y_test, predictions)
r2 = r2_score(Y_test, predictions)

print(f"Mean Absolute Error: {round(mae, 3)}")
print(f"Mean Squared Error: {round(mse, 3)}")
print(f"R2 Score : {round(r2, 3)}")

print("\n")

rel_error = mae/Y_test.mean()
print(f"Relative error : {round(rel_error, 3)}")
print(f"Accuracy : {round((1-rel_error), 3)*100}%")